In [1]:
import os
import glob
import inro.modeller as m
import inro.emme.matrix as _matrix
import numpy as np
import pandas as pd
mm = m.Modeller()    
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
eb = mm.emmebank

In [4]:
def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)
def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
scen = 404

#Matrix naming for inputs

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS TOTAL TRANSIT peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS TOTAL TRANSIT offpeak"

# distance
sov_dist_am = "skim.am_peak.auto.sov.dist"
sov_dist_md = "skim.midday.auto.sov.dist"

# travel time
active_time = "skim.static.bike.time"

sov_time_am = "skim.am_peak.auto.sov.time"
hov2_time_am = "skim.am_peak.auto.hov2.time"
hov3_time_am = "skim.am_peak.auto.hov3.time"
sov_time_md = "skim.midday.auto.sov.time"
hov2_time_md = "skim.midday.auto.hov2.time"
hov3_time_md = "skim.midday.auto.hov3.time"

#costs
sov_toll_am = "skim.am_peak.auto.sovt.toll_cost"
hov2_toll_am = "skim.am_peak.auto.hov2t.toll_cost"
hov3_toll_am = "skim.am_peak.auto.hov3t.toll_cost"

In [5]:
#Delete all matrices
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"
local_fare_mat = "mf180"
transit_aux_time = "mf6"
truck_cost = "mf190"

clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak,local_fare_mat,transit_aux_time, truck_cost],"FULL")

matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Short List\BAU\BAU_ShortList_APTG_2051_GGH_MTP_V106A3_CS222_20201127"

matrix_list = {
    sov_demand_pk : "",
    hov2_demand_pk : "",
    hov3_demand_pk : "",
    bike_demand_pk : "",
    walk_demand_pk : "",
    transit_demand_pk : "",
    sov_demand_offpk : "",
    hov2_demand_offpk : "",
    hov3_demand_offpk : "",
    active_time : "",
    sov_dist_am : "",
    sov_time_am: "",
    hov2_time_am: "",
    hov3_time_am : "",
    sov_time_md: "",
    hov2_time_md: "",
    hov3_time_md : "",
    bike_demand_offpk:"",
    walk_demand_offpk:"",
    transit_demand_offpk:"",
    sov_toll_am :"",
    hov2_toll_am :"",
    hov3_toll_am :"",
    sov_dist_md:""}


for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                  matrix_description = matrix_name)
import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",scen)

exp_pk_pers_trips = "(" + matrix_list[hov2_demand_pk] \
        + "+" + matrix_list[hov3_demand_pk] \
        + "+" + matrix_list[sov_demand_pk] \
        + "+" + matrix_list[bike_demand_pk] + "+" + matrix_list[transit_demand_pk] \
        + "+" + matrix_list[walk_demand_pk] + ")"

exp_offpk_pers_trips = "(" + matrix_list[hov2_demand_offpk] \
        + "+" + matrix_list[hov3_demand_offpk] \
        + "+" + matrix_list[sov_demand_offpk] \
        + "+" + matrix_list[bike_demand_offpk] + "+" + matrix_list[transit_demand_offpk] \
        + "+" + matrix_list[walk_demand_offpk] + ")"

pk_pers_trips = create_matrix(matrix_id="pk_pers_trips",
                        matrix_name="pk.pers.trips",
                        matrix_description="pk.pers.trips",
                        default_value=0)
spec = {
       "type": "MATRIX_CALCULATION",
       "result": pk_pers_trips,
       "expression": exp_pk_pers_trips,
         "constraint": {
    "by_value": None,
    "by_zone": None}
        }
matrix_calculator(spec)

offpk_pers_trips = create_matrix(matrix_id="offpk_pers_trips",
                        matrix_name="offpk.pers.trips",
                        matrix_description="offpk.pers.trips",
                        default_value=0)
spec = {
       "type": "MATRIX_CALCULATION",
       "result": offpk_pers_trips,
       "expression": exp_offpk_pers_trips,
         "constraint": {
    "by_value": None,
    "by_zone": None}
        }
matrix_calculator(spec)


trips_am_peak_auto3_toll_nh
trips_am_peak_auto2_nt_hov
trips_am_peak_auto3_nt_nh
trips_am_peak_auto2_toll_hov
trips_am_peak_auto1_nt
trips_am_peak_auto3_toll_hov
trips_am_peak_auto2_toll_nh
trips_am_peak_auto2_nt_nh
trips_am_peak_auto1_toll
trips_am_peak_auto3_nt_hov
Imported matrix mf63
Imported matrix mf67
Imported matrix mf60
Imported matrix mf66
Imported matrix mf62
Imported matrix mf65
Imported matrix mf68
Imported matrix mf61
Imported matrix mf64
Imported matrix mf59
Finished Importing Matrices


{'average': 0.164845,
 'expression_type': 'FULL',
 'maximum': 1051.734741,
 'maximum_at': {'destination': 8828, 'origin': 8828},
 'minimum': 0.0,
 'minimum_at': {'destination': 1034, 'origin': 1001},
 'num_evaluations': 13003236,
 'result_type': 'FULL',
 'sum': 2143521.75}